In [0]:
dbutils.widgets.text("param_data_source", "")
var_data_source = dbutils.widgets.get("param_data_source")

In [0]:
dbutils.widgets.text("param_file_date", "2021-03-21") # based on the name of the subfolder in blob storage
var_file_date = dbutils.widgets.get("param_file_date")

### Read the JSON file 

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/utils"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
name_schema = StructType([StructField("forename", StringType(), True),
                          StructField("surname", StringType(), True)])

drivers_schema = StructType([StructField("driverId", IntegerType(), False), 
                              StructField("driverRef", StringType(), True),
                              StructField("number", IntegerType(), True),
                              StructField("code", StringType(), True),
                              StructField("name", name_schema),
                              StructField("dob", StringType(), True),
                              StructField("nationality", StringType(), True),
                              StructField("url", StringType(), True)
                            ])


drivers_df = spark.read \
    .schema(drivers_schema) \
    .json(f"{RAW_FOLDER_PATH}/{var_file_date}/drivers.json")   


### rename columns and add new column

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, date_trunc, from_utc_timestamp, lit

In [0]:
modified_drivers_df = add_ingestion_date(drivers_df.withColumnRenamed("driverRef", "driver_ref") \
                                .withColumnRenamed("driverId", "driver_id") \
                                .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))) \
                                .withColumn("data_source", lit(var_data_source)) \
                                .withColumn("file_date", lit(var_file_date))


### drop the unwanted column

In [0]:
final_drivers_df = modified_drivers_df.drop(col("url"))

### write to parquet file

In [0]:
final_drivers_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.drivers")

In [0]:
%sql 
SELECT COUNT(1) FROM f1_processed.drivers

In [0]:
dbutils.notebook.exit("Success")